## **Area-average over complete surface**

The area-averaged WSS is defined as:

\begin{equation}
    \left \langle WSS_A \right \rangle (t)
    =
    \frac{1}{A}
    \int_{S}{}
        \left\| \vec{WSS} (\vec{x},t) \right\| 
        \mathrm{d}S   
\end{equation}

where $A$ is the area of the surface $S$. Note that $\left \langle WSS_A \right \rangle (t)$ will be a function of time. This metric is used to numerically verify the results with respect to mesh independence analysis. Physically, it is equivalent to an overall traction due to the tangential viscous stress on the surface. 

According to authors like Roach and Ferziger, in a Richardson extrapolation-like analysis, it is possible to use integral or derivatives of the priumitive variables to analyze mesh convergence. Ideally, one should analyze both primitive variables in specific locations and quadratures of these that are the variables that will be used in the study being carried-out. First, we will integrate the variables on the chosen surface and in the sequence, we must multiply the magWSS resultant field by the density $\rho = 1056 kg/m^3$, because the loaded 'wallShearComponent' in OpenFOAM is calculated per specific mass unity. 

<hr>

## **OSI and RRT**

In this part of this notebook we will created a sequence of commands to calculate the *oscillatory shear index* (OSI) of a WSS field in an aneurysm surface, from CFD results and using ParaView's Temporal Statistics filter. The function also computes time-averaged data of the WSS, necessary in the OSI definition, and also the *relative residence time (RRT)*.

The OSI is defined as:

\begin{equation}
    OSI(\vec{x})
    =
    \frac{1}{2}
    \left(
        1 
        -
        \frac{
            \left \| 
            \displaystyle\int_0^{T}
            {
                \overrightarrow{WSS} (\vec{x},t)
                \mathrm{d}t
            }
            \right \|
        }{
            \displaystyle\int_0^{T}
            {
                \left \| \vec{WSS} (\vec{x},t) \right \|
                \mathrm{d}t
            }
        }
    \right)
\end{equation}

it measures how the WSS direction changes in time. The RRT is defined as:

\begin{equation}
    RRT(\vec{x})
    =
    \frac{1}{
            \frac{1}{T}
                \left \| 
                    \displaystyle\int_0^{T}
                    {
                        \vec{WSS} (\vec{x},t)
                        \mathrm{d}t
                    }
                \right \|
      }
     =
     \frac{1}{(1 - 2OSI)\overline{WSS}}
\end{equation}


<hr>

## **Computing integrals over space (aneurysm surface) of time-averaged WSS**

The following code computes the integrals over the aneurysm surface of the time-averaged WSS, already calculated in a single surface by, for example, using the OSI function, above. Therefore, the code only clips the aneurysm neck in the same points as used above to compute the time-dependent WSS surface integrals. 

The procedure then will provide a representative value of WSS used for comparison with other publications. With the time-averaged defined as:

\begin{equation}
    \overline{WSS}(\vec{x})
    =
    \frac{1}{T}
    \int_0^{T}
    {
        \left \| \vec{WSS} (\vec{x},t) \right \|
        \mathrm{d}t
    }
\end{equation}

which is called "wallShearComponent_magnitude_average". This is, therefore, a single spatial field defined on the aneurysm surface. Two quantities are extracted from it: 

    - the aneurysm surface-averaged: 
\begin{equation}
    WSS_{av}
    =
    \frac{1}{A_a}
    \int_{S_a}{}
        \overline{WSS}(\vec{x})
    \mathrm{d}S_a   
\end{equation}

    - the maximum value of $\overline{WSS}(\vec{x})$:
\begin{equation}
    WSS_{max} 
    =
    \max_{\vec{x} \in S_a} \overline{WSS}(\vec{x})
\end{equation}

<hr>

## **Post-process aneurysm surface integrals over time**

The procedure below calculates the integral over the aneurysm surface $S_a$ of the WSS magnitude, which is a time-dependent integral:

\begin{equation}
    \left \langle \vec{WSS} \right \rangle (t)
    =
    \frac{1}{A_a}
    \int_{S_a}{}
        \left \| \vec{WSS} \right \|
    \mathrm{d}S_a   
\end{equation}

which is a measure of the total traction due to the tangential component of viscous traction on the aneurysm surface $S_a$. 

The procedure loads an OpenFOAM case of the aneurysm simulation and another surface must be loaded where is stored the array to clip the aneurysm neck: this can be performed in VMTK on a surface where the time averaged field were computed (time-average of WSS magnitude, magnitude of time-averaged WSS vector and OSI). 

<hr>

## Normalized area of *low WSS*

The metric *low WSS area (LSA)* (Jou et al. 2008 - Jou2008a), defined as the area of the aneurysm wall exposed to a WSS below 10% of the mean parent arterial WSS (or another reference value) and then normalized by the dome surface area, $A_a$. This metric is usually calculated for the time-averaged WSS, $\overline{WSS}(\vec{x})$, however I will define it also for the WSS magnitude at the peak systole.

The mean parent artery WSS must be calculated on the anterior genu of the internal carotid siphon for lateral aneurysms on the ICA for example, but, in general, works studying the LSA compute this averaged value on the aneurysm parent artery by averaging the WSS vector over a strip defined on the vessel wall. This reference is used because it is considered as a normal WSS level in the aneurysm region. This definition was also used originally by Jou et al. (2008).

In this notebook, we will calculate the LSA based on a generic *lowWSSValue*, which can be chosen by the user.


In [1]:
#%%writefile aneurysms.py

import paraview.simple as pv
import numpy as np

In [2]:
def aneurysm_area(timeAveragedSurface, 
                  aneurysmNeckArrayName,
                  neckIsoValue=0.5):
    """ Docstring """
    
    clipAneurysmNeck = pv.Clip()
    clipAneurysmNeck.Input = timeAveragedSurface
    clipAneurysmNeck.ClipType = 'Scalar'
    clipAneurysmNeck.Scalars  = ['POINTS', aneurysmNeckArrayName]
    clipAneurysmNeck.Invert   = 1   # gets portion smaller than neckIsoValue
    clipAneurysmNeck.Value    = neckIsoValue  # based on the definition of field ContourScalars
    clipAneurysmNeck.UpdatePipeline()

    # Finaly we integrate over Sa 
    integrateOverAneurysm = pv.IntegrateVariables()
    integrateOverAneurysm.Input = clipAneurysmNeck
    integrateOverAneurysm.UpdatePipeline()

    return integrateOverAneurysm.CellData.GetArray("Area").GetRange()[0] # aneurysm area    
    

In [3]:
def area_averaged_wss(case):
    """ Function that calculates the area-averaged WSS
        for a surface where the wall shear stress field
        is defined. The function takes its input an 
        OpenFOAM case reader with the surface and fields.
        It automatically selects the wallShearComponent field
        and the surface wall. It returns an array with time
        in one column and the result in the other. 
    """
    # Update arrays to be used:
    # - wallShearComponent
    

    case.CellArrays  = ['wallShearComponent']

    # And select the surface where integration will be carried out
    case.MeshRegions = ['wall']

    # Get time-steps values
    timeSteps = np.array(case.TimestepValues)
    # Update time-step
    case.UpdatePipeline()

    # Integrate WSS on the wall
    computeWallArea = pv.IntegrateVariables()
    computeWallArea.Input = case
    computeWallArea.UpdatePipeline()
    
    # Get area of surface, in m2
    wallArea = computeWallArea.CellData.GetArray('Area').GetRange()[0]

    areaAveragedWSSList = []
    for timeStep in timeSteps:
        # Calculate WSS magnitude
        # Instantiate calculater filter
        calcMagWSS = pv.Calculator()
        calcMagWSS.Input = case
        calcMagWSS.ResultArrayName = 'magWSS'
        calcMagWSS.Function = '1056*mag(wallShearComponent)'
        calcMagWSS.AttributeType = 'Cell Data'
        calcMagWSS.UpdatePipeline(time=timeStep)

        # Integrate WSS on the wall
        integrateWSS = pv.IntegrateVariables()
        integrateWSS.Input = calcMagWSS
        integrateWSS.UpdatePipeline(time=timeStep)

        # Instantiate calculater filter
        areaAveragedWSS = pv.Calculator()
        areaAveragedWSS.Input = integrateWSS
        areaAveragedWSS.ResultArrayName = 'areaAveragedWSS'
        areaAveragedWSS.Function = 'magWSS/'+str(wallArea)
        areaAveragedWSS.AttributeType = 'Cell Data'
        areaAveragedWSS.UpdatePipeline(time=timeStep)
        areaAveragedWSSList.append([timeStep, 
                                    areaAveragedWSS.CellData.GetArray('areaAveragedWSS').GetRange()[0]])

    return np.asarray(areaAveragedWSSList)


def osi(ofCaseFile, 
        timeIndexRange, 
        outputFileName,
        timeStep=0.01,
        density=1056.0, # kg/m3
        wssFieldName='wallShearComponent', 
        patchName='wall'):
    """ 
        Function to calculate the oscillatory shear index and
        other time integrals variables of WSS over a time inter-
        val [Ti,Tf] indentified by time-step indices. The method
        (based o VTK), ignores the time-step size and 
        consider uniform time stepping (so, if the time-step is 
        large, the resulting fields may be very different if a va-
        riable time-step would be considered). The OSI field is 
        defined as:
        
            OSI = 0.5*( 1 - norm2(int WSS dt) / int norm2(WSS) dt)
            
        where "int" implies the integral over time between two
        instants t1 and t2 (usually for a cardiac cycle, therefore 
        [t1, t2] = [Ti, Tf]) and norm2 is the L2 norm of an Eucli-
        dean vector field; WSS is the wall shear stress defined on 
        the input surface. Since this function use OpenFOAM data, 
        please specify the density considered.
        
        Input args:
        - OpenFOAM case file (str): name of OpenFOAM .foam case;
        - wssFieldName (str, optional): string containing the name 
            of the wall shear stress field (default="wallShearComp-
            onent");
        - patchName (str, optional): patch name where to calculate 
            the OSI (default="wall");
        - timeIndexRange (list): list of initial and final time-
            steps indices limits of the integral [Ti, Tf];
        - outputFileName (str): file name for the output file with 
            osi field (must be a .vtp file).
        - blood density (float, optional): default 1056.0 kg/m3
    """
    case = pv.OpenFOAMReader(FileName=ofCaseFile)

    # First we define only the field that are going to be used: the WSS on the aneurysm wall
    case.CellArrays = [wssFieldName]
    case.MeshRegions = [patchName]
    case.Createcelltopointfiltereddata = 0

    # Multiplying WSS per density
    densityTimesWSS = pv.Calculator()
    densityTimesWSS.Input = case
    densityTimesWSS.AttributeType   = 'Cell Data'
    densityTimesWSS.ResultArrayName = "WSS"
    densityTimesWSS.Function = str(density)+"*"+wssFieldName
    densityTimesWSS.UpdatePipeline()

    # Calculating the magnitude of the wss vector
    calcMagWSS = pv.Calculator()
    calcMagWSS.Input = densityTimesWSS
    calcMagWSS.AttributeType   = 'Cell Data'
    calcMagWSS.ResultArrayName = densityTimesWSS.ResultArrayName+"_magnitude"

    ## Get WSS field name
    wss = densityTimesWSS.ResultArrayName
    calcMagWSS.Function = "mag("+wss+")"
    calcMagWSS.UpdatePipeline()

    # Extract desired time range
    timeInterval = pv.ExtractTimeSteps()
    timeInterval.Input = calcMagWSS
    timeInterval.SelectionMode = "Select Time Range"
    timeInterval.TimeStepRange = timeIndexRange #[99, 199] # range in index
    timeInterval.UpdatePipeline()

    # Period given by time-steps
    period = (timeInterval.TimeStepRange[1] - timeInterval.TimeStepRange[0])*timeStep

    # Now compute the temporal statistics
    # filter computes the average values of all fields
    calcAvgWSS = pv.TemporalStatistics()

    calcAvgWSS.Input = timeInterval
    calcAvgWSS.ComputeAverage = 1
    calcAvgWSS.ComputeMinimum = 0
    calcAvgWSS.ComputeMaximum = 0
    calcAvgWSS.ComputeStandardDeviation = 0
    calcAvgWSS.UpdatePipeline()

    # Calculates OSI
    calcOSI = pv.Calculator()
    calcOSI.Input = calcAvgWSS
    calcOSI.ResultArrayName = 'OSI'
    calcOSI.AttributeType = 'Cell Data'

    # Getting fields:
    # - Get the average of the vector WSS field
    avgVecWSS = calcAvgWSS.CellData.GetArray(wss+"_average").GetName()
    # - Get the average of the magnitude of the WSS field 
    avgMagWSS = calcAvgWSS.CellData.GetArray(calcMagWSS.ResultArrayName+"_average").GetName()

    calcOSI.Function = "0.5*( 1 - ( mag( "+avgVecWSS+" ) )/"+avgMagWSS+" )"
    calcOSI.UpdatePipeline()

    # Compute Relative Residance Time
    calcRRT = pv.Calculator()
    calcRRT.Input = calcOSI
    calcRRT.ResultArrayName = 'RRT'
    calcRRT.AttributeType   = 'Cell Data'
    calcRRT.Function        = str(period)+"/mag("+avgVecWSS+")"
    calcRRT.UpdatePipeline()

    # Final processing of surface: merge blocks
    # and get surface for triangulation
    mergeBlocks = pv.MergeBlocks()
    mergeBlocks.Input = calcRRT
    mergeBlocks.UpdatePipeline()

    extractSurface = pv.ExtractSurface()
    extractSurface.Input = mergeBlocks
    extractSurface.UpdatePipeline()

    triangulate = pv.Triangulate()
    triangulate.Input = extractSurface
    triangulate.UpdatePipeline()

    pv.SaveData(outputFileName,triangulate)

    
def wss_statistics(timeAveragedSurface, aneurysmNeckArrayName, neckIsoValue=0.5):
    """
        Computes surface-averaged and maximum value 
        of time-averaged WSS for an aneurysm surface.
        Input is a PolyData surface with the averaged
        fields and the aneurysm neck contour field. 
        Return list with aneurysm area, WSSav and 
        WSSmax.
    """
    clipAneurysmNeck = pv.Clip()
    clipAneurysmNeck.Input = timeAveragedSurface
    clipAneurysmNeck.ClipType = 'Scalar'
    clipAneurysmNeck.Scalars  = [aneurysmNeckArrayName]
    clipAneurysmNeck.Invert   = 1   # gets smaller portion
    clipAneurysmNeck.Value    = neckIsoValue  # based on the definition of field ContourScalars
    clipAneurysmNeck.UpdatePipeline()

    # Finaly we integrate over Sa 
    integrateOverAneurysm = pv.IntegrateVariables()
    integrateOverAneurysm.Input = clipAneurysmNeck
    integrateOverAneurysm.UpdatePipeline()

    aneurysmArea = integrateOverAneurysm.CellData.GetArray("Area").GetRange()[0] # aneurysm area

    WSSav = integrateOverAneurysm.CellData.GetArray("WSS_magnitude_average").GetRange()[0]/aneurysmArea # averaged
    WSSmax = clipAneurysmNeck.CellData.GetArray("WSS_magnitude_average").GetRange()[1] # maximum value
    WSSmin = clipAneurysmNeck.CellData.GetArray("WSS_magnitude_average").GetRange()[0] # minimum value
    return [aneurysmArea, WSSav, WSSmax, WSSmin]


def area_averaged_wss_aneurysm(ofCaseFile,aneurysmClipSurface,aneurysmNeckArrayName,density=1056.0):
    """
        Function to compute surface integrals of 
        WSS over an aneurysm surface. It takes the 
        OpenFOAM case file and an extra surface where 
        it is stored a field with the aneurysm neck 
        line loaded as a ParaView PolyData surface.
        To my knowledge, it is important that the sur-
        face with thye neck line array be the same as 
        the wall surface of the OpenFOAM case, i.e.
        they are the same mesh.
    """

    # Clip original aneurysm surface in the neck line
    clipAneurysmNeck = pv.Clip()
    clipAneurysmNeck.Input = aneurysmClipSurface
    clipAneurysmNeck.ClipType = 'Scalar'
    clipAneurysmNeck.Scalars  = ['POINTS', aneurysmNeckArrayName]
    clipAneurysmNeck.Invert   = 1   # gets smaller portion
    clipAneurysmNeck.Value    = 0.5  # based on the definition of field ContourScalars
    clipAneurysmNeck.UpdatePipeline()
    
    integrateWSS = pv.IntegrateVariables()
    integrateWSS.Input = clipAneurysmNeck
    integrateWSS.UpdatePipeline()

    # Get area of surface, in m2
    aneurysmArea = integrateWSS.CellData.GetArray("Area").GetRange()[0]
    
    # Read OpenFOAM data and process the WSS
    # to get its magnitude
    ofData = pv.OpenFOAMReader(FileName=ofCaseFile)

    # Update arrays to be used:
    ofData.CellArrays = ['wallShearComponent']
    ofData.SkipZeroTime = 1
    # And select the surface where integration will be carried out
    ofData.MeshRegions = ['wall']

    # Get time-steps values
    timeSteps = np.array(ofData.TimestepValues)

    # Update time-step
    ofData.UpdatePipeline()

    # Triangulate data to coincide with
    triangulate = pv.Triangulate()
    triangulate.Input = ofData
    triangulate.UpdatePipeline()

    # Compute magnitude of WSS in each cell of the aneurysm surface
    magWSS = pv.Calculator()
    magWSS.Input = triangulate
    magWSS.ResultArrayName = 'magWSS'
    magWSS.Function = str(density)+'*mag('+triangulate.CellData.GetArray('wallShearComponent').Name+')'
    magWSS.AttributeType = 'Cell Data'
    magWSS.UpdatePipeline()

    # Resample OpenFOAM data to clipped aneeurysm surface
    resampleDataset = pv.ResampleWithDataset()
    resampleDataset.Input  = magWSS
    resampleDataset.Source = clipAneurysmNeck
    resampleDataset.PassCellArrays  = 1
    resampleDataset.PassPointArrays = 1
    resampleDataset.UpdatePipeline()

    # Since all fields in ResampleWithDataSet filter 
    # are interpolated to points, therefore
    # apply point data to cell data fielte
    pointToCellData = pv.PointDatatoCellData()
    pointToCellData.Input = resampleDataset
    pointToCellData.UpdatePipeline()

    areaAveragedWSSList = []

    # # Iterate over time-steps to compute time dependent variables
    # # only on the aneurysm surface: mag of WSS over time 
    for timeStep in timeSteps[-100:-1]: # get last cycle only
        # Integrate WSS on the wall
        integrateWSS = pv.IntegrateVariables()
        integrateWSS.Input = pointToCellData
        integrateWSS.UpdatePipeline(time=timeStep)

        # Instantiate calculater filter
        areaAveragedWSS = pv.Calculator()
        areaAveragedWSS.Input = integrateWSS
        areaAveragedWSS.ResultArrayName = 'areaAveragedWSS'
        areaAveragedWSS.Function = integrateWSS.CellData.GetArray('magWSS').Name+'/'+str(aneurysmArea)
        areaAveragedWSS.AttributeType = 'Cell Data'
        areaAveragedWSS.UpdatePipeline(time=timeStep)

        areaAveragedWSSList.append([timeStep, 
                                    areaAveragedWSS.CellData.GetArray('areaAveragedWSS').GetRange()[0]])

    return np.asarray(areaAveragedWSSList)

In [4]:
def lsa_wss_av(timeAveragedSurface, 
               aneurysmNeckArrayName,
               lowWSSValue, 
               clipAneurysmNeck=0.5):

    # Clip aneurysm surface
    clipAneurysmNeck = pv.Clip()
    clipAneurysmNeck.Input = timeAveragedSurface
    clipAneurysmNeck.ClipType = 'Scalar'
    clipAneurysmNeck.Scalars  = ['POINTS', aneurysmNeckArrayName]
    clipAneurysmNeck.Invert   = 1   # gets smaller portion
    clipAneurysmNeck.Value    = neckIsoValue  # based on the definition of field ContourScalars
    clipAneurysmNeck.UpdatePipeline()
    
    # Integrate to get aneurysm surface area
    integrateOverAneurysm = pv.IntegrateVariables()
    integrateOverAneurysm.Input = clipAneurysmNeck
    integrateOverAneurysm.UpdatePipeline()
        
    aneurysmArea = integrateOverAneurysm.CellData.GetArray('Area').GetRange()[0] # m2

    # Clip the aneurysm surface in the lowWSSValue
    # ang gets portion smaller than it
    clipLSA = pv.Clip()
    clipLSA.Input = clipAneurysmNeck
    clipLSA.ClipType = 'Scalar'
    clipLSA.Scalars  = ['CELLS', 'WSS_magnitude_average']
    clipLSA.Invert   = 1   # gets portion smaller than the value
    clipLSA.Value    = lowWSSValue
    clipLSA.UpdatePipeline()

    # Integrate to get area of lowWSSValue
    integrateOverLSA = pv.IntegrateVariables()
    integrateOverLSA.Input = clipLSA
    integrateOverLSA.UpdatePipeline()
    
    area = integrateOverLSA.CellData.GetArray('Area')
    if area == None:
        lsaArea = 0.0
    else:
        lsaArea = integrateOverLSA.CellData.GetArray('Area').GetRange()[0]
    
    return lsaArea/aneurysmArea

In [6]:
def lsa_instant(ofDataFile, 
               aneurysmClipSurface,
               aneurysmNeckArrayName,
               lowWSSValue, 
               peakSystoleInstant,
               clipAneurysmNeck=0.5,
               density=1056.0):
    """ Docstring """
    
    # Clip original aneurysm surface in the neck line
    clipAneurysmNeck = pv.Clip()
    clipAneurysmNeck.Input = aneurysmClipSurface
    clipAneurysmNeck.ClipType = 'Scalar'
    clipAneurysmNeck.Scalars  = ['POINTS', aneurysmNeckArrayName]
    clipAneurysmNeck.Invert   = 1   # gets smaller portion
    clipAneurysmNeck.Value    = 0.5  # based on the definition of field ContourScalars
    clipAneurysmNeck.UpdatePipeline()
    
    integrateWSS = pv.IntegrateVariables()
    integrateWSS.Input = clipAneurysmNeck
    integrateWSS.UpdatePipeline()

    # Get area of surface, in m2
    aneurysmArea = integrateWSS.CellData.GetArray("Area").GetRange()[0]

    # Read OpenFOAM data and process the WSS
    # to get its magnitude
    ofData = pv.OpenFOAMReader(FileName=ofDataFile)

    # Update arrays to be used:
    ofData.CellArrays = ['wallShearComponent']
    ofData.SkipZeroTime = 1
    # And select the surface where integration will be carried out
    ofData.MeshRegions = ['wall']
    # Update time-step
    ofData.UpdatePipeline(time=peakSystoleInstant)

    # Triangulate data to coincide with time averaged surface
    triangulate = pv.Triangulate()
    triangulate.Input = ofData
    triangulate.UpdatePipeline(time=peakSystoleInstant)

    # Compute magnitude of WSS in each cell of the aneurysm surface
    magWSS = pv.Calculator()
    magWSS.Input = triangulate
    magWSS.ResultArrayName = 'magWSS'
    magWSS.Function = str(density)+'*mag('+triangulate.CellData.GetArray('wallShearComponent').Name+')'
    magWSS.AttributeType = 'Cell Data'
    magWSS.UpdatePipeline(time=peakSystoleInstant)

    # Resample OpenFOAM data to clipped aneeurysm surface
    resampleDataset = pv.ResampleWithDataset()
    resampleDataset.Input  = magWSS
    resampleDataset.Source = clipAneurysmNeck
    resampleDataset.PassCellArrays  = 1
    resampleDataset.PassPointArrays = 1
    resampleDataset.UpdatePipeline(time=peakSystoleInstant)

    # Clip the aneurysm surface in the lowWSSValue
    # ang gets portion smaller than it
    clipLSA = pv.Clip()
    clipLSA.Input = resampleDataset
    clipLSA.ClipType = 'Scalar'
    clipLSA.Scalars  = ['POINTS', 'magWSS']
    clipLSA.Invert   = 1   # gets portion smaller than the value
    clipLSA.Value    = lowWSSValue
    clipLSA.UpdatePipeline(time=peakSystoleInstant)

    # Integrate to get area of lowWSSValue
    integrateOverLSA = pv.IntegrateVariables()
    integrateOverLSA.Input = clipLSA
    integrateOverLSA.UpdatePipeline(time=peakSystoleInstant)

    area = integrateOverLSA.CellData.GetArray('Area')
    if area == None:
        lsaArea = 0.0
    else:
        lsaArea = integrateOverLSA.CellData.GetArray('Area').GetRange()[0]

    return lsaArea/aneurysmArea

In [7]:
# This calculation depends on the WSS defined only on the 
# parent artery surface. I thimk the easiest way to com-
# pute that is by drawing the artery contour in the same 
# way as the aneurysm neck is beuild. So, I will assume
# in this function that the surface is already cut to in-
# clude only the parent artery portion and that includes 

def wss_parent_vessel(parentVesselSurface,parentArteryArrayName,parentArteryIsoValue=0.5):
    """ Docstrings """
    
    clipParentArtery = pv.Clip()
    clipParentArtery.Input = parentVesselSurface
    clipParentArtery.ClipType = 'Scalar'
    clipParentArtery.Scalars  = ['POINTS', parentArteryArrayName]
    clipParentArtery.Invert   = 1   # gets smaller portion
    clipParentArtery.Value    = parentArteryIsoValue  # based on the definition of field ContourScalars
    clipParentArtery.UpdatePipeline()

    # Finaly we integrate over Sa 
    integrateOverArtery = pv.IntegrateVariables()
    integrateOverArtery.Input = clipParentArtery
    integrateOverArtery.UpdatePipeline()

    parentArteryArea = integrateOverArtery.CellData.GetArray("Area").GetRange()[0] # aneurysm area

    return integrateOverArtery.CellData.GetArray("WSS_magnitude_average").GetRange()[0]/parentArteryArea

In [8]:
def prettyDict(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            prettyDict(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

<hr>

# Test cells

In [ ]:
# # Get aneurysm surface area
# aneurysmArea = aneurysm_area(aneurysmClipSurface, aneurysmNeckArrayName, 0.5)

# # Read OpenFOAM data and process the WSS
# # to get its magnitude
# ofData = pv.OpenFOAMReader(FileName=ofCaseFile)

# # Update arrays to be used:
# ofData.CellArrays = ['wallShearComponent']
# ofData.SkipZeroTime = 1
# # And select the surface where integration will be carried out
# ofData.MeshRegions = ['wall']
# # Update time-step
# ofData.UpdatePipeline(time=peakSystoleInstant)

# # Triangulate data to coincide with time averaged surface
# triangulate = pv.Triangulate()
# triangulate.Input = ofData
# triangulate.UpdatePipeline(time=peakSystoleInstant)

# # Compute magnitude of WSS in each cell of the aneurysm surface
# magWSS = pv.Calculator()
# magWSS.Input = triangulate
# magWSS.ResultArrayName = 'magWSS'
# magWSS.Function = str(1056)+'*mag('+triangulate.CellData.GetArray('wallShearComponent').Name+')'
# magWSS.AttributeType = 'Cell Data'
# magWSS.UpdatePipeline(time=peakSystoleInstant)

# # Resample OpenFOAM data to clipped aneeurysm surface
# resampleDataset = pv.ResampleWithDataset()
# resampleDataset.Input  = magWSS
# resampleDataset.Source = clipAneurysmNeck
# resampleDataset.PassCellArrays  = 1
# resampleDataset.PassPointArrays = 1
# resampleDataset.UpdatePipeline(time=peakSystoleInstant)

# # Clip the aneurysm surface in the lowWSSValue
# # ang gets portion smaller than it
# clipLSA = pv.Clip()
# clipLSA.Input = resampleDataset
# clipLSA.ClipType = 'Scalar'
# clipLSA.Scalars  = ['POINTS', 'magWSS']
# clipLSA.Invert   = 1   # gets portion smaller than the value
# clipLSA.Value    = lowWSSValue
# clipLSA.UpdatePipeline(time=peakSystoleInstant)

# # Integrate to get area of lowWSSValue
# integrateOverLSA = pv.IntegrateVariables()
# integrateOverLSA.Input = clipLSA
# integrateOverLSA.UpdatePipeline(time=peakSystoleInstant)

# lsaArea = integrateOverLSA.CellData.GetArray('Area').GetRange()[0]

# print(lsaArea/aneurysmArea)

In [ ]:
data = pv.OpenFOAMReader(FileName=pathToFoamCase+foamFileName)

In [ ]:
data.CellData.GetNumberOfArrays()

In [ ]:
meshIntermediateWSS = area_averaged_wss(data)
meshIntermediateWSS

In [ ]:
import matplotlib.pyplot as plt
# import seaborn as sb

In [ ]:
plt.style.use('classic')

In [ ]:
#%matplotlib widget
fig = plt.figure()

plt.plot(timeSteps[-100:-1], lsa[:], 'b')

plt.xlabel('Time')
plt.ylabel('Area-averaged WSS (Pa)')

plt.grid()
plt.show()

In [ ]:
osi("/home/iagolessa/foam/iagolessa-4.0/run/aneurysms/unruptured/fluidFlow/Newtonian/case13/mesh1500k/mesh1500k.foam", [99,199], "/home/iagolessa/foam/iagolessa-4.0/run/aneurysms/unruptured/fluidFlow/Newtonian/case13/mesh1500k/case13_timeAveragedWSSandOSI_Newtonian.vtp")

<hr>

From the time averaged files created above, area-averaging them to get WSS statistics

In [9]:
import os, glob
import matplotlib.pyplot as plt
# import seaborn as sb
# plt.style.use('classic')

In [10]:
fileList = glob.glob("/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/*.vtp")
fileList

['/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case4_timeAveragedWSSandOSI_Newtonian.vtp',
 '/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case11_timeAveragedWSSandOSI_Casson.vtp',
 '/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case13_timeAveragedWSSandOSI_Newtonian.vtp',
 '/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case4_timeAveragedWSSandOSI_CarreauYasuda.vtp',
 '/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case11_timeAveragedWSSandOSI_CarreauYasuda.vtp',
 '/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case2_timeAveragedWSSandOSI_CarreauYasuda.vtp',
 '/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case13_timeAveragedWSSandOSI_Casson.vtp',
 '/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case11_tim

In [11]:
# Definitions

rootPath = "/home/iagolessa/foam/iagolessa-4.0/run/aneurysms/"
models   = ["Newtonian", "Casson", "CarreauYasuda"]
rupturedCases   = ["case1", "case2", "case4", "case19"]
unrupturedCases = ["case11", "case13", "case17"]
aneurysmNeckArrayName = "AneurysmNeckContour"
states = ["ruptured", "unruptured"]

In [12]:
# Compute low WSS limit based on WSS in parent artery 
lowWSSValues = {key: {} for key in rupturedCases+unrupturedCases}
lowWSSValues

{'case1': {},
 'case2': {},
 'case4': {},
 'case19': {},
 'case11': {},
 'case13': {},
 'case17': {}}

In [13]:
for surfaceFile in fileList:
    # Get file case identifier
    identifier = os.path.basename(surfaceFile).strip(".vtp").split("_")
    case  = identifier[0]
    model = identifier[2]
    print("Processing: ", case+' '+model)
    timeAveragedSurface = pv.XMLPolyDataReader(FileName=surfaceFile)
    
    lowWSSValues[case][model] = 0.10*wss_parent_vessel(timeAveragedSurface,'ParentArteryContour')

Processing:  case4 Newtonian
Processing:  case11 Casson
Processing:  case13 Newtonian
Processing:  case4 CarreauYasuda
Processing:  case11 CarreauYasuda
Processing:  case2 CarreauYasuda
Processing:  case13 Casson
Processing:  case11 Newtonian
Processing:  case1 Newtonian
Processing:  case1 CarreauYasuda
Processing:  case4 Casson
Processing:  case17 Casson
Processing:  case13 CarreauYasuda
Processing:  case19 CarreauYasuda
Processing:  case17 CarreauYasuda
Processing:  case19 Newtonian
Processing:  case1 Casson
Processing:  case2 Casson
Processing:  case17 Newtonian
Processing:  case2 Newtonian
Processing:  case19 Casson


In [14]:
prettyDict(lowWSSValues)

case1
	Newtonian
		0.8500538801669972
	CarreauYasuda
		0.8513323658696305
	Casson
		0.953421366146945
case2
	CarreauYasuda
		0.45549216655910113
	Casson
		0.4954320254430064
	Newtonian
		0.45154799084291625
case4
	Newtonian
		0.34681711525372455
	CarreauYasuda
		0.3761720540731499
	Casson
		0.4099578061859597
case19
	CarreauYasuda
		1.5003667106956922
	Newtonian
		1.5060634732298277
	Casson
		1.6849183123714484
case11
	Casson
		1.1045281452326066
	CarreauYasuda
		0.9946950911058283
	Newtonian
		0.9956364781410173
case13
	Newtonian
		1.3097293738713145
	Casson
		1.6474415079891473
	CarreauYasuda
		1.4860340597260053
case17
	Casson
		1.5961453420371001
	CarreauYasuda
		1.4370109697108888
	Newtonian
		1.4437844994020341


In [15]:
# ofCaseFile = "/home/iagolessa/foam/iagolessa-4.0/run/aneurysms/unruptured/fluidFlow/Newtonian/case11/mesh1500k/mesh1500k.foam"
# timeAveragedData = "/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case11_timeAveragedWSSandOSI_Newtonian.vtp"
aneurysmNeckArrayName = "AneurysmNeckContour"
# lowWSSValue = 1.5 # Pascal
neckIsoValue = 0.5
peakSystoleInstant = 1.1

In [16]:
# aneurysmClipSurface = pv.XMLPolyDataReader(FileName=timeAveragedData)

In [ ]:
# ofData = pv.OpenFOAMReader(FileName=ofCaseFile)
# timeSteps = np.array(ofData.TimestepValues)
# timeSteps

In [ ]:
lsa = []
for time in timeSteps[-25:-1]:
    lsa.append(lsa_instant(ofCaseFile,aneurysmClipSurface,aneurysmNeckArrayName,1.5,time))

In [17]:
aneurysmsCases = ["case1", "case2", "case4ruptured", "case19", "case4unruptured", "case11", "case13", "case17"]
lsaCases = {key: {} for key in aneurysmsCases}
lsaCases

{'case1': {},
 'case2': {},
 'case4ruptured': {},
 'case19': {},
 'case4unruptured': {},
 'case11': {},
 'case13': {},
 'case17': {}}

In [ ]:
for surfaceFile in fileList:
    # Get file case identifier
    identifier = os.path.basename(surfaceFile).strip(".vtp").split("_")
    case  = identifier[0]
    model = identifier[2]
    print("Processing: ", surfaceFile)
    timeAveragedSurface = pv.XMLPolyDataReader(FileName=surfaceFile)
    
#     Check aneurysm state base on case
    if case in rupturedCases:
        aneurysmState = states[0]

        if case == "case4": 
            meshLevel = "mesh2000k"
        else:
            meshLevel = "mesh1500k"
            
    elif case in unrupturedCases:
        aneurysmState = states[1]
        
        if case == "case17": 
            meshLevel = "mesh1100k"
        else:
            meshLevel = "mesh1500k"
        
        
#     Build foam file path
    foamFile = rootPath+aneurysmState+"/fluidFlow/"+model+'/'+case+'/'+meshLevel+'/'+meshLevel+'.foam'
    print("Processing:", foamFile)

#     Load surface
    timeAveragedSurface = pv.XMLPolyDataReader(FileName=surfaceFile)
    
    ofData = pv.OpenFOAMReader(FileName=foamFile)
    timeSteps = np.array(ofData.TimestepValues)
    
    # Compute array statistics
    # Check if reaches case4 (two aneurysms)
    if case == "case4":
        for state in states:
            print('Processing:', state)
#             Compute statistics in each aneurysm
#             try:
            timeAveragedSurface = pv.XMLPolyDataReader(FileName=surfaceFile)
            LSAav = lsa_wss_av(timeAveragedSurface,
                               state+aneurysmNeckArrayName,
                               lowWSSValue=lowWSSValues[case][model])

            LSAOverTime = []
            for time in timeSteps[-100:-1]:
                LSAOverTime.append(lsa_instant(foamFile,
                                                timeAveragedSurface,
                                                state+aneurysmNeckArrayName,
                                                lowWSSValue=lowWSSValues[case][model],
                                                peakSystoleInstant=time))

            lsaCases[case+state][model] = {"LSAav" : LSAav,
                                           "LSAt"  : LSAOverTime}

#             except AttributeError:
#                 print("Attribute Error for case", surfaceFile)
#             except:
#                 print("Error for case", surfaceFile)  
    else:        
#         try:
        timeAveragedSurface = pv.XMLPolyDataReader(FileName=surfaceFile)
        LSAav = lsa_wss_av(timeAveragedSurface,
                           aneurysmNeckArrayName,
                           lowWSSValue=lowWSSValues[case][model])

        LSAOverTime = []
        for time in timeSteps[-100:-1]:
            LSAOverTime.append(lsa_instant(foamFile,
                                    timeAveragedSurface,
                                    aneurysmNeckArrayName,
                                    lowWSSValue=lowWSSValues[case][model],
                                    peakSystoleInstant=time))

        lsaCases[case][model] = {"LSAav" : LSAav,
                                 "LSAt"  : LSAOverTime}
            
#         except AttributeError:
#             print("Attribute Error for case", surfaceFile)
#         except:
#             print("Error for case", surfaceFile) 

Processing:  /home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/case4_timeAveragedWSSandOSI_Newtonian.vtp
Processing: /home/iagolessa/foam/iagolessa-4.0/run/aneurysms/ruptured/fluidFlow/Newtonian/case4/mesh2000k/mesh2000k.foam
Processing: ruptured
Processing: unruptured


In [ ]:
prettyDict(lsaCases)

In [ ]:
# Calculate statistics for all files above

for surfaceFile in fileList[2:3]:
    # Get file case identifier
    identifier = os.path.basename(surfaceFile).strip(".vtp").split("_")
    case  = identifier[0]
    model = identifier[2]

    # Check aneurysm state base on case
    if case in rupturedCases:
        aneurysmState = states[0]

        if case == "case4": 
            meshLevel = "mesh2000k"
        else:
            meshLevel = "mesh1500k"
            
    elif case in unrupturedCases:
        aneurysmState = states[1]
        
        if case == "case17": 
            meshLevel = "mesh1100k"
        else:
            meshLevel = "mesh1500k"
        
        
    # Build foam file path
    foamFile = rootPath+aneurysmState+"/fluidFlow/"+model+'/'+case+'/'+meshLevel+'/'+meshLevel+'.foam'
    print("Processing:", foamFile)

    # Load surface
    timeAveragedSurface = pv.XMLPolyDataReader(FileName=surfaceFile)
    
    # Compute array statistics
    # Check if reaches case4 (two aneurysms)
    if case == "case4":
        for state in states:
            # Compute statistics in each aneurysm
            try:
                timeAveragedSurface = pv.XMLPolyDataReader(FileName=surfaceFile)
                stat = wss_statistics(timeAveragedSurface, 
                                      state+aneurysmNeckArrayName)
                
                wssTime = area_averaged_wss_aneurysm(foamFile, 
                                                     timeAveragedSurface, 
                                                     state+aneurysmNeckArrayName)
                
                statistics[case+state][model] = {"Area"  : stat[0],
                                                 "WSSav" : stat[1],
                                                 "WSSmax": stat[2],
                                                 "WSSt"  : wssTime[:,1]}            
            except AttributeError:
                print("Attribute Error for case", surfaceFile)
            except:
                print("Error for case", surfaceFile)  
    else:        
        try:
            stat = wss_statistics(timeAveragedSurface, 
                              aneurysmNeckArrayName)
                
            wssTime = area_averaged_wss_aneurysm(foamFile, 
                                                 timeAveragedSurface, 
                                                 aneurysmNeckArrayName)
            
            statistics[case][model] = {"Area"  : stat[0],
                                       "WSSav" : stat[1],
                                       "WSSmax": stat[2],
                                       "WSSt"  : wssTime[:,1]}  
            
        except AttributeError:
            print("Attribute Error for case", surfaceFile)
        except:
            print("Error for case", surfaceFile) 

In [ ]:
import json
# Write data
writeDir = "/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/processedData/"

In [ ]:
# Tranforming array to list
statisticsBck = statistics
for key in ["case13"]:
    for model in ["Newtonian"]:
        statisticsBck[key][model]["WSSt"] = statistics[key][model]["WSSt"].tolist()

for case in statistics.keys():
    with open(writeDir+'statistics'+case+'.json', 'w') as fp:
        json.dump(statistics[case], 
                  fp, 
                  sort_keys=True, 
                  indent=4)

In [ ]:
# Load already calculated data
for case in statistics.keys():
    with open(writeDir+'statistics'+case+'.json', 'r') as fp:
        statistics[case] = json.load(fp)

In [ ]:
prettyDict(statistics)

In [ ]:
arrangement = [(0, 0, "case1"), 
               (0, 1, "case2"),
               (0, 2, "case4ruptured"),
               (0, 3, "case19"),
               (1, 0, "case4unruptured"),
               (1, 1, "case11"),
               (1, 2, "case13"),
               (1, 3, "case17"),]

In [ ]:
#%matplotlib widget
time = np.arange(0,0.99,0.01)

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(21,10))

for row,column,case in arrangement:
    axes[row,column].plot(time, statistics[case]["Newtonian"]["WSSt"][:], '-b', label="Newtonian")
    axes[row,column].plot(time, statistics[case]["Casson"]["WSSt"][:], '-g', label="Casson")
    axes[row,column].plot(time, statistics[case]["CarreauYasuda"]["WSSt"][:], '-r', label="CarreauYasuda")

    axes[row,column].set_title(case, fontweight="bold")
    axes[row,column].set_xlabel('Time (s)')
    axes[row,column].set_ylabel('Area-averaged WSS (Pa)')
    axes[row,column].legend()
    axes[row,column].grid()
    
plt.show()
fig.savefig("/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/processedData/areaAveragedWSSOverTimePerCase.png", bbox_inches='tight')

In [ ]:
colors = plt.cm.rainbow(np.linspace(0,1,len(statistics.keys())))
colors

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(27,10))

colors = plt.cm.rainbow(np.linspace(0,1,len(statistics.keys())))

for case, color in zip(statistics.keys(), colors):
    axes[0].plot(time, statistics[case]["Newtonian"]["WSSt"][:], color=color, label=case)
    axes[1].plot(time, statistics[case]["Casson"]["WSSt"][:], color=color, label=case)
    axes[2].plot(time, statistics[case]["CarreauYasuda"]["WSSt"][:], color=color, label=case)
    
for index,model in zip(np.arange(0,3),models):
    axes[index].set_title(model+" model", fontweight="bold")
    axes[index].set_xlabel('Time (s)')
    axes[index].set_ylabel('Area-averaged WSS (Pa)')
    axes[index].legend()
    axes[index].grid()

plt.show()
fig.savefig("/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/processedData/areaAveragedWSSOverTimePerModel.png", bbox_inches='tight')

In [ ]:
differences = {key: {} for key in statistics.keys()}
differences

In [ ]:
xlabelsWSSav  = []
xlabelsWSSmax = []

for key in differences.keys():
    differences[key]['normErrAvCasson']  = round(100.0*(statistics[key]['Casson']['WSSav'] - statistics[key]['Newtonian']['WSSav'])/statistics[key]['Newtonian']['WSSav'], 2)
    differences[key]['normErrMaxCasson'] = round(100.0*(statistics[key]['Casson']['WSSmax'] - statistics[key]['Newtonian']['WSSmax'])/statistics[key]['Newtonian']['WSSmax'],2)
    
    differences[key]['normErrAvCY']  = round( 100.0*(statistics[key]['CarreauYasuda']['WSSav'] - statistics[key]['Newtonian']['WSSav'])/statistics[key]['Newtonian']['WSSav'], 2)
    differences[key]['normErrMaxCY'] = round( 100.0*(statistics[key]['CarreauYasuda']['WSSmax'] - statistics[key]['Newtonian']['WSSmax'])/statistics[key]['Newtonian']['WSSmax'], 2)
    
    xlabelsWSSav.append(key+"\n Casson_Err= "+str(differences[key]['normErrAvCasson'])+'%\n CY_Err= '+str(differences[key]['normErrAvCY'])+'%') 
    xlabelsWSSmax.append(key+"\n Casson_Err= "+str(differences[key]['normErrMaxCasson'])+'%\n CY_Err= '+str(differences[key]['normErrMaxCY'])+'%') 

In [ ]:
prettyDict(differences)

In [ ]:
bwidth=0.15

# Instantiate subplots
fig = plt.figure(figsize=(20,8))
ax1  = plt.axes()

X = np.arange(8)

ax1.bar(X + 0.00, [statistics[case]["Newtonian"]["WSSav"] for case in statistics.keys()], color = 'b', width=bwidth, label="Newtonian")
ax1.bar(X + 0.15, [statistics[case]["Casson"]["WSSav"]    for case in statistics.keys()], color = 'g', width=bwidth, label="Casson")
ax1.bar(X + 0.30, [statistics[case]["CarreauYasuda"]["WSSav"] for case in statistics.keys()], color = 'r', width=bwidth, label="CarreauYasuda")

ax1.set_ylabel('WSSav (Pa)')
ax1.legend()
ax1.grid()
ax1.set_axisbelow(True)

plt.xticks(X+0.15, xlabelsWSSav, fontsize='large')

plt.show()
fig.savefig("/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/processedData/timeAveragedWSSav.png", bbox_inches='tight')

In [ ]:
# Instantiate subplots
fig = plt.figure(figsize=(20,10))
ax2 = plt.axes()

X = np.arange(8)

ax2.bar(X + 0.00, [statistics[case]["Newtonian"]["WSSmax"] for case in statistics.keys()], color = 'b', width=bwidth, label="Newtonian")
ax2.bar(X + 0.15, [statistics[case]["Casson"]["WSSmax"]    for case in statistics.keys()], color = 'g', width=bwidth, label="Casson")
ax2.bar(X + 0.30, [statistics[case]["CarreauYasuda"]["WSSmax"] for case in statistics.keys()], color = 'r', width=bwidth, label="CarreauYasuda")

ax2.set_ylabel('WSSmax (Pa)')
ax2.legend()
ax2.grid()
ax2.set_axisbelow(True)

plt.xticks(X+0.15, xlabelsWSSmax, fontsize='large')

plt.show()
fig.savefig("/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/results/timeAveragedData/processedData/timeAveragedWSSmax.png", bbox_inches='tight')